In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import sys
import difflib
sys.path.append("../")
from utils import fidelity_check

In [3]:
screenshot_similarity = json.load(open("../record_replay/screenshots/onload_screenshot_similarity.json", 'r'))
screenshot_similarity = {s['directory']: s['screenshot_similarity'] for s in screenshot_similarity for s in screenshot_similarity}

In [6]:
def check_fidelity(dirr):
    screenshot_simi = screenshot_similarity[dirr.split('/')[-1]]
    live_html = open(f"{dirr}/live.html").read()
    archive_html = open(f"{dirr}/archive.html").read()
    live_element = json.load(open(f"{dirr}/live_elements.json"))
    archive_element = json.load(open(f"{dirr}/archive_elements.json"))
    my_fidelity = fidelity_check.verify(live_html, live_element, archive_html, archive_element)
    print(screenshot_simi >= 1, my_fidelity)

# check_fidelity('testcases/theftaz.azag.gov_1')

#### Test difflib

In [27]:
e1 = fidelity_check.htmlElement({'xpath': '/a/b', 'text': 'hello'})
e2 = fidelity_check.htmlElement({'xpath': '/a/b', 'text': 'hello'})
e1 == e2

/a/b /a/b True


True

## Check archive differences

In [4]:
dirr = 'testcases/www.nato.int_1'

live_element = json.load(open(f"{dirr}/live_elements.json"))
archive_element = json.load(open(f"{dirr}/archive_elements.json"))
# fidelity_check.verify( live_element, archive_element)
live_unique, archive_unique = fidelity_check.diff(live_element, archive_element)
print(len(live_unique), len(archive_unique))
for i in live_unique:
    print("live:\n", i)
for i in archive_unique:
    print("archive:\n", i)

live_unique number [47]
archive_unique number [36]
1 1
live:
             <div class="shelf--slider" role="toolbar">
              <ul class="js-dynamic-mediaslider cf slick-initialized slick-slider js-slider-initialized" data-slider-items="1|3|5">
                <button type="button" data-role="none" class="slick-next slick-arrow" aria-label="Next" role="button" aria-disabled="false">
                  Next
                <div aria-live="polite" class="slick-list draggable">
                  <div class="slick-track" style="opacity: 1; width: 2200px; left: 0px;" role="listbox">
                    <li class="js-slider-item slick-slide" data-slick-index="9" aria-hidden="true" style="width: 205px;" tabindex="-1" role="option" aria-describedby="slick-slide29">
                        <a href="https://www.nato.int/cps/en/natohq/115204.htm" title="NATO-Russia: setting the record straight" tabindex="-1">
                          <a href="https://www.nato.int/cps/en/natohq/115204.htm" tit

In [99]:
# * Generate testcase
diff = {"live": [], "archive": []}
for lu in live_unique:
    first_line = lu.strip().split('\n')[0]
    diff['live'].append({"root": first_line, "numlines": len(lu.strip().split('\n'))})
for au in archive_unique:
    first_line = au.strip().split('\n')[0]
    diff['archive'].append({"root": first_line, "numlines": len(au.strip().split('\n'))})
print(json.dumps(diff, indent=4))

{
    "live": [
        {
            "root": "<div class=\"media media--blazy media--bundle--slideshow-carousel media--slick media--responsive media--image is-b-loaded\" style=\"\" tabindex=\"0\">",
            "numlines": 2
        },
        {
            "root": "<div class=\"ytp-title-channel\">",
            "numlines": 1
        },
        {
            "root": "<div class=\"ytp-title-channel\">",
            "numlines": 1
        }
    ],
    "archive": [
        {
            "root": "<div class=\"media media--blazy media--bundle--slideshow-carousel media--slick media--loading is-b-loading media--responsive media--image\" style=\"\" tabindex=\"0\">",
            "numlines": 2
        },
        {
            "root": "<button class=\"ytp-watch-later-button ytp-button ytp-show-watch-later-title\" data-tooltip-opaque=\"true\" title=\"Watch later\" aria-label=\"Watch later\">",
            "numlines": 7
        }
    ]
}
